In [3]:
import fasttext
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import os
import random

import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Input, Dropout, Lambda, Concatenate

# Have to download the stopwords
# nltk.download('stopwords')

In [308]:
# Get the fasttext model (we are using the largest one they offer [600B tokens])
fasttext_model = fasttext.load_model('models/crawl-300d-2M-subword.bin')

## Data Processsing and Organization
Here, all we really want to do is prepare the data for training. This is **only** the data from **Gold Standard** This includes:
* Simplifying the original data
* Normalizing the data 
* Balancing the positive and negative examples
* Creating the embedding representations that will actually get fed into the neural network

In [13]:
def remove_stop_words(phrase):
    # Creates the stopwords
    to_stop = stopwords.words('english')
    punctuation = "!”#$%&’()*+,-./:;<=>?@[\]^_`{|}~ "
    for c in punctuation:
        to_stop.append(c)

    to_stop.append('null')
    
    for punc in punctuation:
        phrase = phrase.replace(punc, ' ')
    
    return ' '.join((' '.join([x for x in phrase.split(' ') if x not in to_stop])).split())


In [32]:
# Organizing and normalizing the data
"""
Essentially, we want to only have three attributes for each training example: title_one, title_two, label
For normalization, we are just going to use the nltk stopwords and punctuation
"""

def preprocessing(orig_data):
    """
    Normalizes the data by getting rid of stopwords and punctuation
    """
    
    # The new names of the columns
    column_names = ['title_one', 'title_two', 'label']
    # A new dataframe for the data we are going to be creating
    norm_computers = pd.DataFrame(columns = column_names)
    # Iterate over the original dataframe (I know it is slow and there are probably better ways to do it)
    for row in orig_data.itertuples():
        title_left = remove_stop_words(row.title_left)
        title_right = remove_stop_words(row.title_right)
        
        # Append the newly created row (title_left, title_right, label) to the new dataframe
        norm_computers = norm_computers.append(pd.DataFrame([[title_left, title_right, row.label]], columns=column_names))
        
    return norm_computers
        

In [33]:
def create_simple_data():
    """
    Creates and saves a simpler version of the original data that only contains the the two titles and the label.
    """
    
    # Get the dataset of computer parts
    computers_df = pd.read_json('data/train/computers_train_xlarge_normalized.json.gz',compression='gzip', lines=True)
    norm_computers = preprocessing(computers_df)
    
    # Save the new normalized and simplified data to a CSV file to load later
    norm_computers.to_csv('data/train/computers_train_xlarge_norm_simple.csv', index=False)

In [34]:
# Create and save the data if the simple and normalized data does not exist
if not os.path.exists('data/train/computers_train_xlarge_norm_simple.csv'):
    create_simple_data()

In [ ]:
# Load the data
computer_df = pd.read_csv('data/train/computers_train_xlarge_norm_simple.csv')

In [ ]:
# See some of the data. There is clearly a separation between the positive and negative examples
computer_df

In [64]:
def create_train_df(df):
    """
    Returns a shuffled dataframe with an equal amount of positive and negative examples
    """
    # Get the positive and negative examples
    pos_df = df.loc[df['label'] == 1]
    neg_df = df.loc[df['label'] == 0]
    
    # Shuffle the data
    pos_df = pos_df.sample(frac=1)
    neg_df = neg_df.sample(frac=1)
    
    # Concatenate the positive and negative examples and 
    # make sure there are only as many negative examples as positive examples
    final_df = pd.concat([pos_df[:min(len(pos_df), len(neg_df))], neg_df[:min(len(pos_df), len(neg_df))]])
    
    # Shuffle the final data once again
    final_df.sample(frac=1)
    return final_df

In [36]:
# Create and save the dataframe with equal numbers of positive and negative examples
# and is shuffled
if not os.path.exists('data/train/computers_train_bal_shuffle.csv'):
    create_train_df(computer_df).to_csv('data/train/computers_train_bal_shuffle.csv', index=False)

In [75]:
final_computer_df = pd.read_csv('data/train/computers_train_bal_shuffle.csv')

In [76]:
final_computer_df

,title_one,title_two,label
0,corsair carbide air 240 windowed,corsair carbide series air 240 cube micro atx ...,1
1,a8 7670k black edition quad core amd cpu fan h...,amd a8 7650k 3 3ghz pccomponentes,1
2,amazonbasics 13 3 inch laptop sleeve black acc...,amazonbasics 13 3 inch laptop sleeve black car...,1
3,eg0146fartr hp 146 gb 6g 10k 2 5 dp sas hdd ne...,eg0146fartr hp 146 gb 6g 10k 2 5 dp sas hdd,1
4,usb 3 0 external adapter cable 2 5 inch hard d...,transcend ssd370 solid state drive ssd 2 5 sat...,0
...,...,...,...
19375,356816 001 ml350t g4p xeon 3 2 2mb 512mb whole...,409159 b21 hp xeon e5345 2 33ghz dl160 g3 new ...,0
19376,buy online samsung 750 evo series 120gb ssd mz...,ssd 750 basic 120 gb tradineur com,1
19377,628061 s21 hp g8 g9 3 tb 6g 7 2k 5 sata sc new...,628061 s21 hp g8 g9 3 tb 6g 7 2k 5 sata sc new...,1
19378,buy online zotac gtx 1060 6gb amp edition grap...,msi nvidia geforce gtx 1080 8gb gaming x rgb g...,0


## Laptop Data Preprocessing
* Normalize the data
* Create negative examples that represent when only a couple of attributes of the laptop data changes

In [51]:
# Load the laptop data
laptop_df = pd.read_csv('data/train/laptops.csv', encoding='latin-1')

In [52]:
laptop_df

,Unnamed: 0,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,2537.45
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298,1316,Lenovo,Yoga 500-14ISK,2 in 1 Convertible,14.0,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i7 6500U 2.5GHz,4GB,128GB SSD,Intel HD Graphics 520,Windows 10,1.8kg,638.00
1299,1317,Lenovo,Yoga 900-13ISK,2 in 1 Convertible,13.3,IPS Panel Quad HD+ / Touchscreen 3200x1800,Intel Core i7 6500U 2.5GHz,16GB,512GB SSD,Intel HD Graphics 520,Windows 10,1.3kg,1499.00
1300,1318,Lenovo,IdeaPad 100S-14IBR,Notebook,14.0,1366x768,Intel Celeron Dual Core N3050 1.6GHz,2GB,64GB Flash Storage,Intel HD Graphics,Windows 10,1.5kg,229.00
1301,1319,HP,15-AC110nv (i7-6500U/6GB/1TB/Radeon,Notebook,15.6,1366x768,Intel Core i7 6500U 2.5GHz,6GB,1TB HDD,AMD Radeon R5 M330,Windows 10,2.19kg,764.00


In [53]:
# This class will be used in order to exchange the different attributes
# to create negative examples
class Attributes():
    company = {'Apple'}
    product = {'MacBook Pro'}
    inches = {'13.3'}
    cpu = {'Intel Core i5 2.3GHz'}
    ram = {'4GB'}
    memory = {'256GB SSD'}
    gpu = {'Intel HD Graphics 520'}
    screen = {'1440x900'}
    
    def get_all_data():
        return {
            'company': Attributes.company,
            'product': Attributes.product,
            'inches': Attributes.inches,
            'cpu': Attributes.cpu,
            'ram': Attributes.ram,
            'memory': Attributes.memory,
            'gpu': Attributes.gpu,
            'screen': Attributes.screen
        }

In [54]:
# Create attribute sets
def create_attribute_sets(df):
    Attributes.company.update([row.Company for row in laptop_df[['Company']].itertuples()])
    Attributes.product.update([row.Product for row in laptop_df[['Product']].itertuples()])
    Attributes.inches.update([str(row.Inches) for row in laptop_df[['Inches']].itertuples()])
    Attributes.cpu.update([row.Cpu for row in laptop_df[['Cpu']].itertuples()])
    Attributes.ram.update([row.Ram for row in laptop_df[['Ram']].itertuples()])
    Attributes.memory.update([row.Memory for row in laptop_df[['Memory']].itertuples()])
    Attributes.gpu.update([row.Gpu for row in laptop_df[['Gpu']].itertuples()])
    Attributes.screen.update([row.ScreenResolution for row in laptop_df[['ScreenResolution']].itertuples()])

create_attribute_sets(laptop_df)

In [120]:
def concatenate_row(row):
    # Note: got rid of everything after the '(' because it has info about the actual specs of the laptop
    # so if we change the specs, we need to fix that too
    return ' '.join([row['Company'], row['Product'].split('(')[0], row['TypeName'], str(row['Inches']), 'inch',  row['ScreenResolution'], row['Cpu'], row['Ram'], 'ram', row['Memory'], row['Gpu']])

In [56]:
# Creates the negative examples for the laptop data
# The laptop_df is the original data, the new_df is the dataframe to append the new data to
# and the attributes are the attributes to swap for the new data
def create_neg_laptop_data(laptop_df, attributes):
    new_column_names = ['title_one', 'title_two', 'label']
    negative_df = pd.DataFrame(columns = new_column_names)
    for row in range(len(laptop_df)):
        # Create a copy of the row for the negative example
        neg_row = laptop_df.iloc[row]
        for attribute_class in attributes:
            # Get the row in the laptop_data
            orig_row = laptop_df.iloc[row]
            
            # Get the attribute that we are trying to change
            attribute_val = orig_row[attribute_class]
            
            # Temporarily value for the new value
            new_val = attribute_val
            
            # Make sure we really get a new attribute
            while new_val == attribute_val:
                new_val = random.sample(Attributes.get_all_data()[attribute_class.lower()], 1)[0]
            
            # Change the value in the neg_row to the new value
            neg_row[attribute_class] = new_val
            
            # Concatenate and normalize the data
            title_one = remove_stop_words(concatenate_row(orig_row).lower())
            title_two = remove_stop_words(concatenate_row(neg_row).lower())
            
            # Append the data to the new df
            negative_df = negative_df.append(pd.DataFrame([[title_one, title_two, 0]], columns=new_column_names))
    
    return negative_df

In [57]:
neg_df = create_neg_laptop_data(laptop_df, attributes=['Cpu', 'Memory', 'Ram', 'Inches', 'Product'])

C:\Users\Jason\miniconda3\envs\tf2\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [58]:
neg_df

,title_one,title_two,label
0,apple macbook pro ultrabook 13 3 inch ips pane...,apple macbook pro ultrabook 13 3 inch ips pane...,0
0,apple macbook pro ultrabook 13 3 inch ips pane...,apple macbook pro ultrabook 13 3 inch ips pane...,0
0,apple macbook pro ultrabook 13 3 inch ips pane...,apple macbook pro ultrabook 13 3 inch ips pane...,0
0,apple macbook pro ultrabook 13 3 inch ips pane...,apple macbook pro ultrabook 18 4 inch ips pane...,0
0,apple macbook pro ultrabook 13 3 inch ips pane...,apple ux410ua gv350t ultrabook 18 4 inch ips p...,0
...,...,...,...
0,asus x553sa xx031t notebook 15 6 inch 1366x768...,asus x553sa xx031t notebook 15 6 inch 1366x768...,0
0,asus x553sa xx031t notebook 15 6 inch 1366x768...,asus x553sa xx031t notebook 15 6 inch 1366x768...,0
0,asus x553sa xx031t notebook 15 6 inch 1366x768...,asus x553sa xx031t notebook 15 6 inch 1366x768...,0
0,asus x553sa xx031t notebook 15 6 inch 1366x768...,asus x553sa xx031t notebook 11 6 inch 1366x768...,0


In [59]:
# Creates the postive examples for the laptop data
# The laptop_df is the original data, the new_df is the dataframe to append the new data to
# and the attributes are the attributes to swap or delete for the new data
def create_pos_laptop_data(laptop_df, rm_attrs, add_attrs):
    new_column_names = ['title_one', 'title_two', 'label']
    pos_df = pd.DataFrame(columns = new_column_names)
    for row in range(len(laptop_df)):
        # Remove the attribute from the new title
        for attr_list in rm_attrs:
            # Create a copy of the row for the negative example
            new_row = laptop_df.iloc[row]
            orig_row = laptop_df.iloc[row]
            for attr in attr_list:
                new_row[attr] = ''
        
            title_one = remove_stop_words(concatenate_row(orig_row).lower())
            title_two = remove_stop_words(concatenate_row(new_row).lower())

            # Occassionally add in the operating system just to switch it up
            if (random.sample([0, 1], 1)):
                for attr in add_attrs:
                    title_two += ' ' + orig_row[attr].lower()

            pos_df = pos_df.append(pd.DataFrame([[title_one, title_two, 1]], columns=new_column_names))

    return pos_df

In [60]:
pos_df = create_pos_laptop_data(laptop_df, rm_attrs = [['Company'], ['TypeName'], ['ScreenResolution'], ['Product'], ['TypeName', 'ScreenResolution']], add_attrs=['OpSys'])

C:\Users\Jason\miniconda3\envs\tf2\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [61]:
pos_df

,title_one,title_two,label
0,apple macbook pro ultrabook 13 3 inch ips pane...,macbook pro ultrabook 13 3 inch ips panel reti...,1
0,apple macbook pro ultrabook 13 3 inch ips pane...,apple macbook pro 13 3 inch ips panel retina d...,1
0,apple macbook pro ultrabook 13 3 inch ips pane...,apple macbook pro ultrabook 13 3 inch intel co...,1
0,apple macbook pro ultrabook 13 3 inch ips pane...,apple ultrabook 13 3 inch ips panel retina dis...,1
0,apple macbook pro ultrabook 13 3 inch ips pane...,apple macbook pro 13 3 inch intel core i5 2 3g...,1
...,...,...,...
0,asus x553sa xx031t notebook 15 6 inch 1366x768...,x553sa xx031t notebook 15 6 inch 1366x768 inte...,1
0,asus x553sa xx031t notebook 15 6 inch 1366x768...,asus x553sa xx031t 15 6 inch 1366x768 intel ce...,1
0,asus x553sa xx031t notebook 15 6 inch 1366x768...,asus x553sa xx031t notebook 15 6 inch intel ce...,1
0,asus x553sa xx031t notebook 15 6 inch 1366x768...,asus notebook 15 6 inch 1366x768 intel celeron...,1


## Laptop Data Concatenation
Create on dataframe and shuffle the data

In [68]:
def create_laptop_data(pos_df, neg_df):
    pos_df.sample(frac=1)
    neg_df.sample(frac=1)
    final_laptop_df = pd.concat([pos_df[:min(len(pos_df), len(neg_df))], neg_df[:min(len(pos_df), len(neg_df))]])
    final_laptop_df = final_laptop_df.sample(frac=1)
    return final_laptop_df

In [69]:
final_laptop_df = create_laptop_data(pos_df, neg_df)

In [70]:
final_laptop_df

,title_one,title_two,label
0,acer chromebook c731 c78g netbook 11 6 inch ip...,acer vivobook max netbook 14 0 inch ips panel ...,0
0,dell vostro 5471 ultrabook 14 0 inch full hd 1...,dell vostro 5471 14 0 inch intel core i5 8250u...,1
0,acer aspire a517 51g notebook 15 6 inch ips pa...,acer aspire a517 51g notebook 15 6 inch intel ...,1
0,asus zenbook ux310uq gl026t ultrabook 13 3 inc...,asus zenbook ux310uq gl026t ultrabook 11 3 inc...,0
0,dell inspiron 7577 gaming 15 6 inch ips panel ...,dell inspiron 7577 gaming 15 6 inch ips panel ...,0
...,...,...,...
0,lenovo ideapad 320 15ikbn notebook 15 6 inch f...,lenovo notebook 15 6 inch full hd 1920x1080 in...,1
0,dell vostro 5568 notebook 15 6 inch full hd 19...,dell vostro 5568 notebook 15 6 inch full hd 19...,0
0,dell latitude 3570 notebook 15 6 inch 1366x768...,dell latitude 3570 notebook 15 6 inch 1366x768...,0
0,dell xps 13 ultrabook 13 3 inch quad hd touchs...,dell xps 13 13 3 inch quad hd touchscreen 3200...,1


## Embeddings Creation Functions
Generates the embeddings and saves them

In [7]:
"""
Definitions of some sizes in the training set
"""
MAX_LEN = 42
EMBEDDING_SHAPE = (300,)
print('MAX_LEN: ' + str(MAX_LEN), 'EMBEDDING_SHAPE: ' + str(EMBEDDING_SHAPE))

MAX_LEN: 42 EMBEDDING_SHAPE: (300,)


In [84]:
"""
Create the numpy files of all the training embedddings
We will have two numpy files:
1. The training/validation/test sets
2. The labels
"""

def create_embeddings(df):
    # Create the numpy arrays for storing the embeddings and labels
    total_embeddings = np.zeros(shape=(len(df), 2, MAX_LEN, EMBEDDING_SHAPE[0]))
    labels = np.zeros(shape=(len(df)))
    
    # I know this is a terrible way of doing this, but iterate over the dataframe
    # and generate the embeddings to add to the numpy array
    for idx, row in enumerate(df.itertuples()):
        for word_idx, word in enumerate(row.title_one.split()):
            total_embeddings[idx, 0, word_idx] = fasttext_model[word]
            
        for word_idx, word in enumerate(row.title_two.split()):
            total_embeddings[idx, 1, word_idx] = fasttext_model[word]
            
        labels[idx] = row.label
        
    return total_embeddings, labels


In [85]:
def save_embeddings(df, embeddings_name, labels_name):
    """
    Saves the embeddings given the embeddings file name and labels file name
    """
    if not os.path.exists('data/numpy_data/' + embeddings_name + '.npy'):
        embeddings, labels = create_embeddings(df)
        with open('data/numpy_data/' + embeddings_name + '.npy', 'wb') as f:
            np.save(f, embeddings)

        with open('data/numpy_data/' + labels_name + '.npy', 'wb') as f:
            np.save(f, labels)

In [86]:
def load_embeddings_and_labels(embeddings_name, labels_name):
    loaded_embeddings = None
    labels = None
    with open('data/numpy_data/' + embeddings_name + '.npy', 'rb') as f:
        loaded_embeddings = np.load(f)
        loaded_embeddings = np.transpose(loaded_embeddings, (1, 0, 2, 3))
    
    with open('data/numpy_data/' + labels_name + '.npy', 'rb') as f:
        labels = np.load(f)
    
    return loaded_embeddings, labels

In [87]:
def get_max_len(df):
    max_len = 0
    for row in df.itertuples():
        if len(row.title_one.split(' ')) > max_len:
            max_len = len(row.title_one.split(' '))
            
        if len(row.title_two.split(' ')) > max_len:
            max_len = len(row.title_two.split(' '))
    
    return max_len

## Saving and Loading Embeddings
Save the embeddings for the different types of data we have

In [112]:
# Concatenate everything
total_data = pd.concat([final_computer_df, final_laptop_df])
total_data = total_data.sample(frac=1)
save_embeddings(final_computer_df, 'bal_computers_embeddings', 'bal_computers_labels')
save_embeddings(final_laptop_df, 'laptop_embeddings', 'laptop_labels')
save_embeddings(total_data, 'all_embeddings', 'all_labels')

In [113]:
embeddings, labels = load_embeddings_and_labels('all_embeddings', 'all_labels')

In [114]:
len(embeddings[0,:])

32410

In [115]:
total_data

,title_one,title_two,label
16663,acer aspire es1 132 p194 business notebook 331...,acer aspire es1 132 p194 business notebook len...,1
0,lenovo ideapad 310 15ikb notebook 15 6 inch fu...,lenovo ideapad 310 15ikb 15 6 inch intel core ...,1
0,hp 250 g6 ultrabook 15 6 inch full hd 1920x108...,hp 250 g6 ultrabook 15 6 inch full hd 1920x108...,0
3086,corsair vengeance led 16gb 2x8gb ddr4pc4 21300...,corsair vengeance red led 16gb 2x8gb ddr4 pc4 ...,1
15990,kingston datatraveler 100 g3 32 gb usb 3 0 dt1...,usb datatraveler 100 g3 3 0 stick 32 gb,1
...,...,...,...
11649,seagate laptop sshd 1 tb internal st1000lm014 ...,wd green wds240g1g0a ssd 240 go sata 6gb garan...,0
0,lenovo ideapad 320 17isk notebook 17 3 inch 16...,lenovo notebook 17 3 inch 1600x900 intel core ...,1
15592,sandisk extreme microsdhc 64gb type 10 acheter...,sandisk extreme microsdhc 64gb type 10 kopen e...,1
10730,dg0146famwl hp 146 gb 6g 10k 2 5 dp sas new pa...,dg0146famwl hp 146 gb 6g 10k 2 5 dp sas hdd ne...,1


In [116]:
X_train1 = embeddings[0, :len(labels) - 4000]
X_train2 = embeddings[1, :len(labels) - 4000]
X_train = np.stack((X_train1, X_train2))
print('Training shape: ' + str(X_train.shape))

X_val1 = embeddings[0, len(labels) - 4000:len(labels) - 2000]
X_val2 = embeddings[1, len(labels) - 4000:len(labels) - 2000]
X_val = np.stack((X_val1, X_val2))
print('Val shape: ' + str(X_val.shape))

X_test1 = embeddings[0, len(labels) - 2000:]
X_test2 = embeddings[1, len(labels) - 2000:]
X_test = np.stack((X_test1, X_test2))
print('Test shape: ' + str(X_test.shape))

Training shape: (2, 28410, 42, 300)
Val shape: (2, 2000, 42, 300)
Test shape: (2, 2000, 42, 300)


In [117]:
Y_train = labels[:len(labels) - 4000]
print('Training labels shape:', str(Y_train.shape))

Y_val = labels[len(labels) - 4000:len(labels) - 2000]
print('Val shape:', str(Y_val.shape))

Y_test = labels[len(labels) - 2000:]
print('Test shape:', str(Y_test.shape))

Training labels shape: (28410,)
Val shape: (2000,)
Test shape: (2000,)


In [118]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

In [119]:
Y_train = convert_to_one_hot(Y_train.astype(np.int32), 2)
Y_val = convert_to_one_hot(Y_val.astype(np.int32), 2)
Y_test = convert_to_one_hot(Y_test.astype(np.int32), 2)

In [135]:
Y_train

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]])

## Model Info

For the model, we are going to use LSTMs with a Constrastive Loss Function 
that will also be used to predict whether the two products are the same 

First, we have to convert the titles to embeddings through FastText before feeding into the LSTM.
The embedding part of this model will not be a layer because:
* The fasttext model would be time consuming and annoying to get to work with an embedding layer in Keras
* The fasttext model is not going to be getting its embeddings optimized, so there is really no point in adding it as an embedding layer

In [8]:
def square_distance(vectors):
    x, y = vectors
    return tf.square(x - y)

def euclidean_dist_out_shape(shapes):
    # Both inputs are fed in, so just use one of them and get the first value in the shape
    shape1, shape2 = shapes
    return (shape1[0],)

def siamese_network(input_shape):
    # Defines our inputs
    left_title = Input(input_shape, dtype='float32')
    right_title = Input(input_shape, dtype='float32')
    
    # The LSTM units
    model = tf.keras.Sequential(name='siamese_model')
    model.add(LSTM(units=256, return_sequences=True, name='lstm_1'))
    model.add(Dropout(rate=0.5))
    model.add(LSTM(units=128, return_sequences=True, name='lstm_2'))
    model.add(Dropout(rate=0.5))
    model.add(LSTM(units=128, name='lstm_3'))
    model.add(Dropout(rate=0.5))
    
    # The dense layers
    model.add(Dense(units=1024, activation='elu', name='dense_1'))
    model.add(Dropout(rate=0.5))
    model.add(Dense(units=512, activation='elu', name='dense_2'))
    
    # Forward propagate through the model to generate the encodings
    encoded_left_title = model(left_title)
    encoded_right_title = model(right_title)

    SquareDistanceLayer = Lambda(square_distance)
    distance = SquareDistanceLayer([encoded_left_title, encoded_right_title])
    
    prediction = Dense(units=2, activation='softmax')(distance)
    # Create and return the network
    siamese_net = tf.keras.Model(inputs=[left_title, right_title], outputs=prediction, name='siamese_network')
    return siamese_net

In [121]:
# Note: for the constrastive loss, because 0 denotes that they are from the same class
# and one denotes they are from a different class, I swaped the (Y) and (1 - Y) terms

def constrastive_loss(y_true, y_pred):
    margin = 2.0
    d = y_pred
    d_sqrt = tf.sqrt(d)
    #tf.print('\nY Pred: ', d, 'Shape: ', tf.shape(d))
    #tf.print('\nY True: ', y_true, 'Shape: ', tf.shape(y_true))
    
    loss = (y_true * d) + ((1 - y_true) * tf.square(tf.maximum(0., margin - d_sqrt)))
    
    #tf.print('\n Constrastive Loss: ', loss, 'Shape: ', tf.shape(loss))
    loss = 0.5 * tf.reduce_mean(loss)
    
    return loss

In [122]:
# Accuracy metric for constrastive loss because values close to 0 are equal and values high are different
# 0.5 is the threshold here
def constrastive_accuracy(y_true, y_pred):
    return tf.reduce_mean(tf.cast(tf.equal(y_true, tf.cast(y_pred < 0.5, y_true.dtype)), y_true.dtype))

In [123]:
def save_model(model, name):
    """
    Saves a model with a particular name
    """
    model.save('models/' + name + '.h5')

In [9]:
model = siamese_network((MAX_LEN, EMBEDDING_SHAPE[0],))
model.summary()

Model: "siamese_network"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 42, 300)]    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 42, 300)]    0                                            
__________________________________________________________________________________________________
siamese_model (Sequential)      (None, 512)          1555968     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 512)          0           siamese_model[0][0]

In [128]:
# Compile the model
lr = 0.001
opt = tf.keras.optimizers.Adam(learning_rate=lr)
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])

In [129]:
# Train the model
model.fit(x=[X_train1, X_train2], y=Y_train, batch_size=64, epochs=50, validation_data=([X_val[0], X_val[1]], Y_val))

Train on 28410 samples, validate on 2000 samples
Epoch 1/50
28410/28410 [==============================] - 441s 16ms/sample - loss: 0.4879 - accuracy: 0.6650 - val_loss: 0.6279 - val_accuracy: 0.5010
Epoch 2/50
28410/28410 [==============================] - 421s 15ms/sample - loss: 0.4301 - accuracy: 0.6939 - val_loss: 0.5686 - val_accuracy: 0.5090
Epoch 3/50
28410/28410 [==============================] - 422s 15ms/sample - loss: 0.4256 - accuracy: 0.6995 - val_loss: 0.5701 - val_accuracy: 0.5100
Epoch 4/50
28410/28410 [==============================] - 423s 15ms/sample - loss: 0.4364 - accuracy: 0.6980 - val_loss: 0.5760 - val_accuracy: 0.5010
Epoch 5/50
28410/28410 [==============================] - 421s 15ms/sample - loss: 0.4338 - accuracy: 0.6993 - val_loss: 0.5702 - val_accuracy: 0.5100
Epoch 6/50
28410/28410 [==============================] - 421s 15ms/sample - loss: 0.4264 - accuracy: 0.7050 - val_loss: 0.5676 - val_accuracy: 0.5210
Epoch 7/50
28410/28410 [=====================

In [130]:
# Test the model
results = model.evaluate([X_test1, X_test2], Y_test, batch_size=16)
print('test loss, test acc: ', results)

2000/2000 [==============================] - 12s 6ms/sample - loss: 0.3420 - accuracy: 0.8870
test loss, test acc:  [0.3420196931362152, 0.887]


In [10]:
# Set the model's name
model_name = '0.2_Softmax-LSTM-50_epochs'

In [ ]:
# Save the model
save_model(model, model_name)

## Manual Testing
Converts titles into embeddings arrays and allow the model to make a prediction

In [11]:
model.load_weights('models/' + model_name + '.h5')

In [346]:
title_one = 'LG OLED77CXPUA 77 Class HDR 4K UHD Smart OLED TV (2020 Model)'
title_two = 'LG OLED77CXPUA 77 Alexa Built-In CX Series 4K Ultra HD Smart OLED TV (2020)'
#title_one = 'Corsair 16GB ram'
#title_two = 'G Skill 32GB ram'
title_one_arr = np.zeros((1, 42, 300))
title_two_arr = np.zeros((1, 42, 300))
title_one = remove_stop_words(title_one.lower())
title_two = remove_stop_words(title_two.lower())

for idx, word in enumerate(title_one.split(' ')):
    title_one_arr[0, idx] = fasttext_model[word]
    
for idx, word in enumerate(title_two.split(' ')):
    title_two_arr[0, idx] = fasttext_model[word]

In [347]:
model.predict([title_one_arr, title_two_arr])

array([[0.34407118, 0.6559288 ]], dtype=float32)